In [1]:
from pathlib import Path

from parcellate.interfaces.qsirecon.loader import load_qsirecon_inputs
from parcellate.interfaces.qsirecon.qsirecon import QSIReconConfig

DATA_ROOT = Path("/media/storage/yalab-dev/qsiprep_test/derivatives/qsirecon")

config = QSIReconConfig(
    input_root=DATA_ROOT,
    output_dir=DATA_ROOT.parent / "qsiparc",
    mask="gm",
)

In [2]:
# from bids import BIDSLayout


# layout = BIDSLayout(
#     DATA_ROOT,
#     validate=False,
#     derivatives=True,
#     config=["bids", "derivatives"],
# )

# filters = {
#         "subject": "CLMC10",
#         "suffix": "dwimap",
#         "extension": ["nii", "nii.gz"],
#     }

# files = layout.get(
#     return_type="object",
#     **filters,
# )


In [3]:
# files = layout.get(
#     return_type="object",
#     desc="modulated",
#     **filters,
# )
# files[0].entities


In [ ]:
from parcellate.interfaces.qsirecon.qsirecon import run_parcellations

res = run_parcellations(config)

2025-11-30 12:33:40,082 [INFO] Loading QSIRecon inputs from /media/storage/yalab-dev/qsiprep_test/derivatives/qsirecon


In [3]:
recon_inputs = load_qsirecon_inputs(root=DATA_ROOT, subjects=["CLMC10"])
recon = recon_inputs[0]

In [4]:
from parcellate.interfaces.qsirecon.planner import plan_qsirecon_parcellation_workflow

plan = plan_qsirecon_parcellation_workflow(recon)

In [5]:
key = next(iter(plan))
# task = plan[key]
# task[0].nifti_path
simple_plan = {k: [v[0]] for k, v in plan.items()}

In [6]:
from parcellate.interfaces.qsirecon.runner import run_qsirecon_parcellation_workflow

simple_result = run_qsirecon_parcellation_workflow(recon, simple_plan, config=config)

In [7]:
result = simple_result[0]

In [14]:
destination = config.output_dir
workflow = result.scalar_map.recon_workflow or "parcellate"
base = destination / f"qsirecon-{workflow}"

subject_dir = base / f"sub-{result.context.subject_id}"
if result.context.session_id:
    subject_dir = subject_dir / f"ses-{result.context.session_id}"

# QSIRecon organizes diffusion derivatives under ``dwi``
output_dir = subject_dir / "dwi" / f"atlas-{result.atlas.name}"

entities: list[str] = [result.context.label]
space = result.atlas.space or result.scalar_map.space
entities.append(f"atlas-{result.atlas.name}")
if space:
    entities.append(f"space-{space}")
if result.atlas.resolution:
    entities.append(f"res-{result.atlas.resolution}")
if result.scalar_map.model:
    entities.append(f"model-{result.scalar_map.model}")
entities.append(f"param-{result.scalar_map.param}")
if result.scalar_map.desc:
    entities.append(f"desc-{result.scalar_map.desc}")


filename = "_".join([*entities, "parc"]) + ".tsv"
out_path = output_dir / filename

In [15]:
result.scalar_map

ScalarMapDefinition(name='noddi-icvf-modulated', nifti_path=PosixPath('/media/storage/yalab-dev/qsiprep_test/derivatives/qsirecon/derivatives/qsirecon-AMICONODDI/sub-CLMC10/ses-202407110849/dwi/sub-CLMC10_ses-202407110849_dir-AP_space-MNI152NLin2009cAsym_model-noddi_param-icvf_desc-modulated_dwimap.nii.gz'), param='icvf', model='noddi', origin='Tissue fraction modulated intracellular volume fraction from NODDI', space='MNI152NLin2009cAsym', desc='modulated', recon_workflow='AMICONODDI')

In [16]:
out_path

PosixPath('/media/storage/yalab-dev/qsiprep_test/derivatives/qsiparc/qsirecon-AMICONODDI/sub-CLMC10/ses-202407110849/dwi/atlas-4S156Parcels/sub-CLMC10_ses-202407110849_atlas-4S156Parcels_space-MNI152NLin2009cAsym_res-01_model-noddi_param-icvf_desc-modulated_parc.tsv')